In [1]:
from multiprocessing.spawn import prepare
from django.shortcuts import render
import os
import datetime
import requests
import shutil
from django.conf import settings
from django.http import HttpResponse, FileResponse
from django.http import StreamingHttpResponse
import json
import re
import mlflow
from mlflow import MlflowClient
from mlflow.tracking import MlflowClient
# from . models import Task_Category
import qrcode
import base64
import io
import re

os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin8888'
os.environ['MLFLOW_TRACKING_URI'] = 'http://195.130.121.234:8085/'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://195.130.121.234:8089/'

client = MlflowClient()

In [2]:
experiments = client.search_experiments()
models = client.search_registered_models()

In [32]:
model = models[0]

In [4]:
print(model.tags.get('Task'))

Classification


In [35]:
model = client.get_registered_model(model.name)

In [53]:
model.latest_versions[0].tags.get('Author')

'Eva Pachetti'

In [43]:
model.latest_versions[0].tags.get('Experiment_ID')

'138'

In [51]:
X = {key:value for key,value in client.get_run(run_id='ca2fb956ae7a4a38bb7d54c380f48d77').data.tags.items()
    if key.startswith('dataset_')}

X

{'dataset_created': '2023-10-13 15:36:14.484195',
 'dataset_modified': '2023-08-01T00:00:00',
 'dataset_type': '.dcm,.yaml,.parquet',
 'dataset_number_of_cases': '50',
 'dataset_md5': 'ce56ead5301457158bc9558160d9b495.dir',
 'dataset_size': '627861366',
 'dataset_nfiles': '1404',
 'dataset_path': 'data',
 'dataset_coregs': 'd033b870c620fd57fa8ffe74b95306bd',
 'dataset_ecrfs_series': '598dc33881b8ad844c1fa1f56df12611',
 'dataset_motcor': '9c7bcc096a554bd355347220e9fa97a2',
 'dataset_segments': 'd59201f7c1f140695d45f732c4b31f5f',
 'dataset_ageHigh': '91',
 'dataset_ageLow': '0',
 'dataset_annotated': 'True',
 'dataset_anonymized': 'True',
 'dataset_contactPoint': 'procancer-i-technical@procancer-i.eu',
 'dataset_description': 'This dataset contains a collection of retrospective patients with confirmed PCa at biopsy and/or prostatectomy.',
 'dataset_deviceManufacturer': 'General Electric, Toshiba, Philips, Quibim, Siemens',
 'dataset_distribution': 'aaaadczjudn6oascvqlaabiaae',
 'dataset_

In [52]:
X["dataset_Name"] = X.pop('dataset_name')
X

{'dataset_created': '2023-10-13 15:36:14.484195',
 'dataset_modified': '2023-08-01T00:00:00',
 'dataset_type': '.dcm,.yaml,.parquet',
 'dataset_number_of_cases': '50',
 'dataset_md5': 'ce56ead5301457158bc9558160d9b495.dir',
 'dataset_size': '627861366',
 'dataset_nfiles': '1404',
 'dataset_path': 'data',
 'dataset_coregs': 'd033b870c620fd57fa8ffe74b95306bd',
 'dataset_ecrfs_series': '598dc33881b8ad844c1fa1f56df12611',
 'dataset_motcor': '9c7bcc096a554bd355347220e9fa97a2',
 'dataset_segments': 'd59201f7c1f140695d45f732c4b31f5f',
 'dataset_ageHigh': '91',
 'dataset_ageLow': '0',
 'dataset_annotated': 'True',
 'dataset_anonymized': 'True',
 'dataset_contactPoint': 'procancer-i-technical@procancer-i.eu',
 'dataset_description': 'This dataset contains a collection of retrospective patients with confirmed PCa at biopsy and/or prostatectomy.',
 'dataset_deviceManufacturer': 'General Electric, Toshiba, Philips, Quibim, Siemens',
 'dataset_distribution': 'aaaadczjudn6oascvqlaabiaae',
 'dataset_

In [86]:
from PIL import Image, ImageDraw, ImageFont
import io

# Create an image with the author's name when qr_code is None
image_width = 300
image_height = 300
background_color = (255, 255, 255)
text_color = (0, 0, 0)

image = Image.new("RGB", (image_width, image_height), background_color)
draw = ImageDraw.Draw(image)

font = ImageFont.truetype("arial.ttf", 30)
text = "author_name"

text_width = draw.textlength(text, font=font)
text_height = font.getbbox(text)[-1]  # get the height of the font
text_x = (image_width - text_width) // 2
text_y = (image_height - text_height) // 2

draw.text((text_x, text_y), text, fill=text_color, font=font,stroke_width=1)

# Convert the image to bytes
img_byte_array = io.BytesIO()
image.save(img_byte_array, format='PNG')
img_byte_array.seek(0)

img = Image.open(img_byte_array)


In [87]:
img.show()

In [82]:
font.getbbox(text)

(0, 6, 179, 34)

In [66]:
font.getlength(text)

63.0